# Importation des packages

In [1]:
# Importation des modules requis
import os
import plotly.express as px
import plotly.offline as pyoff
import plotly.graph_objs as go
import plotly.express as px
import plotly.io as pio
pio.renderers.default = 'iframe'
import pandas as pd
from pandas.tseries.offsets import *
import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from matplotlib import colors
from matplotlib.collections import LineCollection
%matplotlib inline
import seaborn as sns
import numpy as np
import warnings
import plotly.express as px
import datetime as dt
import plotly.graph_objs as go
from datetime import timedelta, datetime
from itertools import product
import tqdm.notebook as tqdm

# Preprocessing
from sklearn.preprocessing import normalize, MinMaxScaler, StandardScaler

# Pipeline
from sklearn.pipeline import make_pipeline, Pipeline
from sklearn.compose import ColumnTransformer

# Modèles
from sklearn.cluster import KMeans
from sklearn import metrics
from sklearn.metrics import silhouette_samples, silhouette_score, homogeneity_score, adjusted_rand_score, adjusted_mutual_info_score

mpl.rcParams["figure.figsize"] = (12, 8)
import warnings
warnings.filterwarnings('ignore')

# Ouverture Dataset

Afin d'éviter de multiplier les étapes de transformation de la data, nous utiliserons le fichier 'clean_data' utilisé dans le Notebook 2 (essais de modélisation).

In [2]:
df_maintenance = pd.read_csv('clean_data.csv')
df_maintenance = df_maintenance.drop('Unnamed: 0', axis = 1).set_index('customer_unique_id')
df_maintenance

,auto,bed_bath_table,computers_accessories,furniture_decor,health_beauty,housewares,other_categories,sports_leisure,telephony,toys,...,mean_price,total_freight_value,mean_payment_sequential,mean_payment_installments,total_payment_value,mean_review_score,mean_freight_ratio,order_purchase_timestamp,recency,frequency
customer_unique_id,,,,,,,,,,,,,,,,,,,,,
0000366f3b9a7992bf8c76cfdf3221e2,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,129.90,12.00,1.0,8.0,141.90,5.0,0.09,2018-05-10 10:56:27,1775,1
0000b849f77a49e4a4ce2b2a4ca5be3f,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,18.90,8.29,1.0,1.0,27.19,4.0,0.44,2018-05-07 11:11:27,1778,1
0000f46a3911fa3c0805444483337064,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,69.00,17.22,1.0,8.0,86.22,3.0,0.25,2017-03-10 21:05:03,2200,1
0000f6ccb0745a6a4b88665a16c9f078,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,25.99,17.63,1.0,4.0,43.62,4.0,0.68,2017-10-12 20:29:41,1984,1
0004aac84e0df4da2b147fca70cf8255,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,180.00,16.89,1.0,6.0,196.89,5.0,0.09,2017-11-14 19:45:42,1951,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
fffcf5a5ff07b0908bd4e2dbc735a684,0.0,0.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0,0.0,...,785.00,497.42,1.0,10.0,4134.84,5.0,0.32,2017-06-08 21:00:36,2110,2
fffea47cd6d3cc0a88bd621562a9d061,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,64.89,19.69,1.0,1.0,84.58,4.0,0.30,2017-12-10 20:07:56,1925,1
ffff371b4d645b6ecea244b27531430a,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,89.90,22.56,1.0,1.0,112.46,5.0,0.25,2017-02-07 15:49:16,2231,1


# Classement temporel

In [3]:
# Tri croissant par date

df_maintenance.sort_values(by = ['order_purchase_timestamp'], ascending = True, inplace = True)

# Création de la dataframe de simulation ('df_simu')

model_cols = ['order_purchase_timestamp',
              'mean_price',
              'mean_payment_installments',
              'total_payment_value',
              'mean_review_score',
              'mean_freight_ratio',
              'recency',
              'frequency'
             ]

df_simu = df_maintenance[model_cols]
df_simu

,order_purchase_timestamp,mean_price,mean_payment_installments,total_payment_value,mean_review_score,mean_freight_ratio,recency,frequency
customer_unique_id,,,,,,,,
2f64e403852e6893ae37485d5fcacdaf,2016-10-03 16:56:50,21.90,1.0,39.09,4.0,0.78,2358,1
61db744d2f835035a5625b59350c6b63,2016-10-03 21:13:36,36.49,1.0,53.73,3.0,0.47,2358,1
8d3a54507421dbd2ce0a1d58046826e0,2016-10-03 22:06:03,119.90,6.0,133.46,1.0,0.11,2358,1
87776adb449c551e74c13fc34f036105,2016-10-03 22:31:31,29.99,4.0,40.95,5.0,0.37,2358,1
7390ed59fa1febbfda31a80b4318c8cb,2016-10-03 22:44:10,128.90,2.0,154.57,5.0,0.20,2358,1
...,...,...,...,...,...,...,...,...
7a22d14aa3c3599238509ddca4b93b01,2018-08-29 12:25:59,63.90,7.0,73.10,1.0,0.14,1664,1
5c58de6fb80e93396e2f35642666b693,2018-08-29 14:18:23,209.00,8.0,1021.92,5.0,0.22,1663,2
7febafa06d9d8f232a900a2937f04338,2018-08-29 14:18:28,45.90,1.0,61.29,5.0,0.34,1663,1


# Split hebdomadaire

In [4]:
df_simu['dates'] = pd.to_datetime(df_simu['order_purchase_timestamp'])
df_simu['dates'] = df_simu['dates'].dt.date
df_simu['Week_ending'] = df_simu['dates'] + Week(weekday = 4)

df_simu

,order_purchase_timestamp,mean_price,mean_payment_installments,total_payment_value,mean_review_score,mean_freight_ratio,recency,frequency,dates,Week_ending
customer_unique_id,,,,,,,,,,
2f64e403852e6893ae37485d5fcacdaf,2016-10-03 16:56:50,21.90,1.0,39.09,4.0,0.78,2358,1,2016-10-03,2016-10-07
61db744d2f835035a5625b59350c6b63,2016-10-03 21:13:36,36.49,1.0,53.73,3.0,0.47,2358,1,2016-10-03,2016-10-07
8d3a54507421dbd2ce0a1d58046826e0,2016-10-03 22:06:03,119.90,6.0,133.46,1.0,0.11,2358,1,2016-10-03,2016-10-07
87776adb449c551e74c13fc34f036105,2016-10-03 22:31:31,29.99,4.0,40.95,5.0,0.37,2358,1,2016-10-03,2016-10-07
7390ed59fa1febbfda31a80b4318c8cb,2016-10-03 22:44:10,128.90,2.0,154.57,5.0,0.20,2358,1,2016-10-03,2016-10-07
...,...,...,...,...,...,...,...,...,...,...
7a22d14aa3c3599238509ddca4b93b01,2018-08-29 12:25:59,63.90,7.0,73.10,1.0,0.14,1664,1,2018-08-29,2018-08-31
5c58de6fb80e93396e2f35642666b693,2018-08-29 14:18:23,209.00,8.0,1021.92,5.0,0.22,1663,2,2018-08-29,2018-08-31
7febafa06d9d8f232a900a2937f04338,2018-08-29 14:18:28,45.90,1.0,61.29,5.0,0.34,1663,1,2018-08-29,2018-08-31


In [5]:
# Suppression des données avant 2017-03, et suppression des colonnes superflues
df_simu = df_simu[df_simu['order_purchase_timestamp'] >= '2017-03-01']
df_simu.drop(['dates', 'order_purchase_timestamp'], axis = 1, inplace = True)

# Création d'un index semaines
date_ = df_simu['Week_ending'].unique()
df_simu = pd.DataFrame(date_).reset_index().rename(columns = {0:'Week_ending', 'index':'Week_index'})\
.merge(df_simu, on = 'Week_ending')

df_simu

,Week_index,Week_ending,mean_price,mean_payment_installments,total_payment_value,mean_review_score,mean_freight_ratio,recency,frequency
0,0,2017-03-03,29.90,1.0,52.18,1.0,0.75,2210,1
1,0,2017-03-03,39.00,1.0,49.96,4.0,0.28,2210,1
2,0,2017-03-03,99.90,1.0,118.03,5.0,0.18,2210,1
3,0,2017-03-03,27.00,1.0,43.05,5.0,0.59,2210,1
4,0,2017-03-03,158.90,1.0,188.18,4.0,0.18,2210,1
...,...,...,...,...,...,...,...,...,...
90246,78,2018-08-31,63.90,7.0,73.10,1.0,0.14,1664,1
90247,78,2018-08-31,209.00,8.0,1021.92,5.0,0.22,1663,2
90248,78,2018-08-31,45.90,1.0,61.29,5.0,0.34,1663,1
90249,78,2018-08-31,24.90,1.0,33.23,3.0,0.33,1663,1


# Modélisation et simulation temporelle

In [6]:
ARI_ = []
scaler = MinMaxScaler()
K = 5

for i in tqdm.tqdm(df_simu['Week_index'].unique()[:-1]):
    
    tmp = df_simu[df_simu['Week_index'] >= i][[x for x in df_simu.columns if x not in ['Week_index', 'Week_ending']]]
    tmp = scaler.fit_transform(tmp)
    model = KMeans(n_clusters=K)
    model.fit(tmp)
    
    tmp = df_simu[df_simu['Week_index'] >= i+1][[x for x in df_simu.columns if x not in ['Week_index', 'Week_ending']]]
    tmp = scaler.transform(tmp)
    model2 = KMeans(n_clusters = K)
    model2.fit(tmp)
    
    kmeans_label2 = model2.labels_
    kmeans_label1 = model.fit(tmp).labels_
    
    ARI_.append(adjusted_rand_score(kmeans_label1, kmeans_label2))

  0%|          | 0/78 [00:00<?, ?it/s]

# Représentation de la précision de modélisation (ARI)

In [9]:
d = {'Weekly ARI'}
ari_table = pd.DataFrame(data = ARI_, columns = d)
ari_table

,Weekly ARI
0,0.966459
1,0.995431
2,0.999921
3,0.999625
4,0.995540
...,...
73,0.982894
74,0.998836
75,1.000000
76,0.998538


In [10]:
pd.options.plotting.backend = "plotly"
fig = ari_table.plot(title = "Evolution de l'ARI sur la modélisation KMeans (hebdomadaire)",
                     labels = dict(index = "Semaine", value = "Precision Score", variable = ""))
fig.show()

# Conclusion

Selon la simualtion effectuée sur la modélisation précédemment sélectionnées, il apparaît qu'une maintenance serait nécessaire au bout de 31 semaines, soit environ 7 mois.  
En effet, le score de précision (ARI) chute en dessous en 0.8 (seuil d'acceptabilité) au delà de cette période (ARI = 0.78).  
Ainsi, un contrat de maintenance peut être envisagé de manière semestrielle.